In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [2]:
from torch.utils.data import TensorDataset, DataLoader

In [3]:
import lightning as pl
from lightning.pytorch.loggers import TensorBoardLogger

import torchmetrics

#### Config for training

defining variables pertaining to how many layers are going to be in the model, what kind of layers, how many batches, in and out features and other details.

In [4]:
# config
max_epochs = 18
learning_rate = 0.005

in_features=837
h1 = 700
h2 = 500
h3 = 200
out_features = 64
batch_size = 512

logging_name = 'linear nn1'
version_tb = f'h{h1}_h{h2}_h{h3}_e{max_epochs}'

project = "chess nn1 linear model 4, my moves, 1 label"
architecture = "Linear network with 2 hidden layers, my moves, 1 label",
dataset = "max rapid games",

### Loading data

In [5]:
# data_training = torch.load('tensor_data/nn1_rapid_games_data_training_09_02_24.pt')
data_training = torch.from_numpy(np.load('np_arrays/nn1_linear_train.npy'))
# print(data_training.shape)
# data_training.to(device)

In [6]:
# data_testing = torch.load('tensor_data/nn1_rapid_games_data_testing_09_02_24.pt')
data_valid = torch.from_numpy(np.load('np_arrays/nn1_linear_valid.npy'))

# print(data_testing.shape)
# data_testing.to(device)

In [7]:
lables_training = torch.from_numpy(np.load('np_arrays/nn1_linear_train_lables.npy'))
# print(lables_training.shape)
# lables_training.to(device)

In [8]:
lables_valid = torch.from_numpy(np.load('np_arrays/nn1_linear_valid_lables.npy'))
# lables_testing = torch.load('tensor_data/nn1_rapid_games_lables_testing_09_02_24.pt')
# print(lables_testing.shape)
# lables_testing.to(device)

In [9]:
# data_training = data_training.half()
# data_testing = data_testing.half()
# lables_training = lables_training.half()
# lables_testing = lables_testing.half()
# lables_training = lables_training.float()
# lables_testing = lables_testing.float()

In [10]:
# converting 64 bit tensors to 32 bit (float functions) and integer numbers (long) for lebel data
data_training = data_training.float()
data_valid = data_valid.float()
lables_training = lables_training.long().view(-1)
lables_valid = lables_valid.long().view(-1)

In [11]:
print(data_training.shape)
print(lables_training.shape)
print(data_valid.shape)
print(lables_valid.shape)

torch.Size([134152, 837])
torch.Size([134152])
torch.Size([33538, 837])
torch.Size([33538])


In [12]:
print(data_training.shape)
print(lables_training.shape)
print(data_valid.shape)
print(lables_valid.shape)

torch.Size([134152, 837])
torch.Size([134152])
torch.Size([33538, 837])
torch.Size([33538])


In [13]:
data_training.shape

torch.Size([134152, 837])

In [14]:
# print(data_valid.dtype)
# print(lables_training.dtype)
# print(lables_valid.dtype)

In [15]:
# setting up a seed
torch.manual_seed(42)

In [16]:
# making a torch dataset object out of the data
training_dataset = TensorDataset(data_training, lables_training)
valid_dataset = TensorDataset(data_valid, lables_valid)

In [17]:
# making a dataloader object out of dataset object (which properly loads data in batches)
data_loader_training = DataLoader(training_dataset, batch_size=batch_size, shuffle=True, num_workers=7)
data_loader_valid = DataLoader(valid_dataset, batch_size=batch_size, num_workers=7)


In [18]:
class Model(pl.LightningModule):

    def __init__(self, in_features=in_features, h1=h1, h2=h2, h3=h3, out_features=64): #out = 64 squares to move to 
        super().__init__()

        self.fc1 = nn.Linear(in_features, h1)
        self.fc2 = nn.Linear(h1, h2)
        # self.fc2 = nn.Linear(h1, h2)
        # self.fc3 = nn.Linear(h2, h3)
        # self.fc3 = nn.Linear(h2, h3)
        self.out = nn.Linear(h2, out_features)

        self.accuracy = torchmetrics.classification.Accuracy(task="multiclass", num_classes=out_features)

    def forward(self, x):
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        # x = F.relu(self.fc3(x))
        x = self.out(x)
        
        return x


    def training_step(self, batch, batch_index):

        x, y = batch
        preds = self.forward(x)
        print(preds)


        loss = F.cross_entropy(preds, y)
        
        accuracy = self.accuracy(preds, y)

        self.log('loss', loss)
        self.log('accuracy', accuracy)

        # self.log('train_acc_step', self.accuracy)

        return loss

    def validation_step(self, batch, batch_index):

        x, y = batch
        preds = self.forward(x)
        loss = F.cross_entropy(preds, y)
        accuracy = self.accuracy(preds, y)
        
        # self.log('train_acc_step', self.accuracy)
        self.log('loss validation', loss)
        self.log('accuracy validation', accuracy)

        return loss
        

    def configure_optimizers(self):

        return torch.optim.Adam(self.parameters(), lr=learning_rate)



In [19]:
model = Model()

In [20]:
# making a Tensorboard logger object
logger = TensorBoardLogger("lightning_logs", name=logging_name, version=version_tb) #tb logs is a directory?


# defining Trainer object and doing actual training.
# setting precision as 16-mixed which may speed up training (as opposed to 64 or 32 bit)
trainer = pl.Trainer(max_epochs=max_epochs, logger=logger, precision='16-mixed')
# trainer = pl.Trainer(max_epochs=20, logger=logger, precision='32')

# trainer.fit(model, data_loader_training)
trainer.fit(model, data_loader_training, data_loader_valid)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type               | Params
------------------------------------------------
0 | fc1      | Linear             | 586 K 
1 | fc2      | Linear             | 350 K 
2 | out      | Linear             | 32.1 K
3 | accuracy | MulticlassAccuracy | 0     
------------------------------------------------
969 K     Trainable params
0         Non-trainable params
969 K     Total params
3.877     Total estimated model params size (MB)


Epoch 0:   0%|          | 0/263 [00:00<?, ?it/s]                           tensor([[-0.0036, -0.0071, -0.0121,  ...,  0.0291, -0.0211,  0.0386],
        [ 0.0040, -0.0174, -0.0228,  ...,  0.0306, -0.0279,  0.0263],
        [ 0.0136, -0.0175, -0.0257,  ...,  0.0255, -0.0373,  0.0247],
        ...,
        [ 0.0051, -0.0219, -0.0172,  ...,  0.0480, -0.0399,  0.0220],
        [ 0.0046, -0.0265, -0.0010,  ...,  0.0480, -0.0580,  0.0142],
        [ 0.0074, -0.0240, -0.0029,  ...,  0.0283, -0.0361,  0.0247]],
       device='cuda:0', dtype=torch.float16, grad_fn=<AddmmBackward0>)
Epoch 0:   0%|          | 1/263 [00:01<05:10,  0.84it/s, v_num=_e18]tensor([[-0.0387, -0.0339,  0.0938,  ...,  0.1935,  0.0600, -0.2566],
        [-0.0438, -0.0131,  0.0853,  ...,  0.1353,  0.0789, -0.2334],
        [-0.0130, -0.0262,  0.0798,  ...,  0.1227,  0.0250, -0.2086],
        ...,
        [-0.0296, -0.0354,  0.0825,  ...,  0.1091,  0.0391, -0.1725],
        [-0.0833, -0.0378,  0.1121,  ...,  0.2446,  0.1201,

`Trainer.fit` stopped: `max_epochs=18` reached.


Epoch 17: 100%|██████████| 263/263 [00:05<00:00, 47.31it/s, v_num=_e18]


### Saving the model to a file for future use
I've saved it earlier in training when the accuracy and loss for validation data were optimal.

In [21]:
torch.save(model.state_dict(), f'models/nn1_h{h1}_h{h2}_epochs_{max_epochs}.pt')